## Prepping bgen ADNI data for analysis

### Preqs:
- conda (on macosx I use `brew install miniconda`)

Create a new environment to work on:

```shell
conda create -n sfsu
conda activate sfsu
```

- bgenix: `conda install -c conda-forge bgenix`
- [pybgen](https://lemieuxl.github.io/pybgen/index.html): 
```shell
conda config --add channels http://statgen.org/wp-content/uploads/Softwares/pybgen
conda install pybgen
```
- pandas: `conda install pandas`

### data

I downloaded three of the files in the [ADNI drive folder](https://drive.google.com/drive/u/0/folders/1whflUk5psModyhGWuecr-DB5ZklszBNp)

- `ADNI_merge.sample`: to get sample ids and use as column headers
- `ADNI_merged_21.bgen`: smaller file to use for testing
- `ADNI_merged_19.bgen`: chr19 data we can use for the activity (APOE gene is on chr19)

I then used the indexing program `bgenix` to index the bgen files (not really necessary)

```shell
bgenix -g ADNI_merged_21.bgen -index
bgenix -g ADNI_merged_19.bgen -index
```


In [1]:
# after it's all done, this is what the directory looks like
!ls -l

total 813328
-rw-r--r--@ 1 corradah  staff      31870 May 21 07:45 ADNI_merged.sample
-rw-r--r--@ 1 corradah  staff  261420612 May 21 08:05 ADNI_merged_19.bgen
-rw-r--r--  1 corradah  staff   13561856 May 21 08:07 ADNI_merged_19.bgen.bgi
-rw-r--r--@ 1 corradah  staff  124140480 May 19 08:45 ADNI_merged_21.bgen
-rw-r--r--  1 corradah  staff    7634944 May 21 07:09 ADNI_merged_21.bgen.bgi
-rw-r--r--  1 corradah  staff       3049 May 21 08:36 prep_data.ipynb


## Dealing with samples

Let's read the `ADNI_merged.sample` file and get sample ids. The file format is described
[here](https://www.well.ox.ac.uk/~gav/qctool/documentation/sample_file_formats.html)


In [7]:
import pandas as pd

# first let's read the header line separately
sample_file = "ADNI_merged.sample"
with open(sample_file, 'r') as f:
    colnames = f.readline().strip().split(' ')
colnames


['ID_1', 'ID_2', 'missing', 'father', 'mother', 'sex', 'plink_pheno']

In [12]:
# now let's read the file itself, let's do 50 lines as a test
sample_df = pd.read_csv(sample_file, header=None, skiprows=2, nrows=50, delimiter=' ')
sample_df.columns = colnames
sample_df.head()

,ID_1,ID_2,missing,father,mother,sex,plink_pheno
0,1,014_S_0520,0.047162,0,0,2,-9
1,2,005_S_1341,0.047162,0,0,2,-9
2,4,012_S_0803,0.047162,0,0,2,-9
3,5,018_S_0055,0.047162,0,0,1,-9
4,6,027_S_0118,0.047162,0,0,1,-9


That seems to work, looks like ID_2 should mean something as an id, let's use that 
as sample names when we prepare the dosage matrix. Let's read the whole thing now.


In [13]:
sample_df = pd.read_csv(sample_file, header=None, skiprows=2, delimiter=' ')
sample_df.columns = colnames
print(f"We read {len(sample_df)} rows")


We read 940 rows


## Handling bgen data

Let's get started reading some info from the bgen file for chr21

In [15]:
from pybgen import PyBGEN

chr21_bgen = "ADNI_merged_21.bgen"

with PyBGEN(chr21_bgen) as bgen:
    print(f"We have {bgen.nb_samples} samples")
    print(f"We have {bgen.nb_variants} variants")

We have 940 samples
We have 177285 variants


Ok, number of samples match, that's a relief... Ok, let's look at dosage for the
first 3 variants

In [18]:
i = 0
with PyBGEN(chr21_bgen) as bgen:
    for info, dosage in bgen:
        if i == 3:
            break
        i += 1
        print(f"variant info: {info}")
        print(f"length of dosage array: {len(dosage)}")
        print(f"first six dosage values: {dosage[:6]}")
        print()

variant info: <Variant rs186660003 chrNA:9419528_A/G>
length of dosage array: 940
first six dosage values: [nan nan nan nan nan nan]

variant info: <Variant rs184649157 chrNA:9576494_G/A>
length of dosage array: 940
first six dosage values: [0.00100708 0.00100708 0.00201416 0.         0.00100708        nan]

variant info: <Variant rs191306879 chrNA:9585758_G/C>
length of dosage array: 940
first six dosage values: [0.0039978 0.        0.        0.0289917 0.        0.       ]



A couple of observations:

- info is a class we'll need to extract pieces of. Class definition [here](https://lemieuxl.github.io/pybgen/_modules/pybgen/pybgen.html#PyBGEN.iter_variant_info)
- chromose info is not retained, so we need to add that ourselves
- there's lot's of nas so we should check for variants that have a lot of nas
- it's not readily apparent what allele was used as reference to compute dosage

Let's deal with info first, and then figure out the reference allele question

In [41]:
# let's make sure we can parse variant info correctly
with PyBGEN(chr21_bgen) as bgen:
    info, _ = bgen.next()
    print(f"rsid: {info.name}, chr: chr21, pos: {int(info.pos)}, ref: {info.a1}, alt: {info.a2}")

rsid: rs186660003, chr: chr21, pos: 9419528, ref: A, alt: G


Ok, that looks good. Regarding the dosage question, let's take the API of
the function [`PyBGEN.get_specific_variant`](https://lemieuxl.github.io/pybgen/_modules/pybgen/pybgen.html#PyBGEN.get_specific_variant)
in good faith and call `a1` the reference allele. Now let's confirm this makes sense...

In [26]:
# let's get dosage values for the second variant
# so we can confirm how they are calculated
with PyBGEN(chr21_bgen) as bgen:
    bgen.next()
    _, dosage = bgen.next()
    returned_dosages = dosage[:10]
returned_dosages

array([0.00100708, 0.00100708, 0.00201416, 0.        , 0.00100708,
              nan, 0.00100708, 0.00100708, 0.01098633, 0.02099609])

In [27]:
# now let's get probabilities for the second variant
# and check this calculation
with PyBGEN(chr21_bgen, probs_only=True) as bgen:
    bgen.next()
    _, probs = bgen.next()
    print(probs)

[[0.99899292 0.00100708 0.        ]
 [0.99899292 0.00100708 0.        ]
 [0.99798584 0.00201416 0.        ]
 ...
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]


Ok, this looks like dosage is wrt to alternate allele, which makes sense. Alright, lets 
compute dosages from the probabilities of the first 10 samples and see what we get.

The dosage calculation, is described [here](https://www.synapse.org/#!Synapse:syn2290704/wiki/64682)



In [30]:
import numpy as np

def mydosage(probs):
    probs = np.array(probs)
    return(probs[2] + 2 * probs[1])

with PyBGEN(chr21_bgen, probs_only=True) as bgen:
    bgen.next()
    _, probs = bgen.next()
    probs = probs[:10]
    calculated_dosages = np.array([ mydosage(p) for p in probs ])
calculated_dosages

array([0.00201416, 0.00201416, 0.00402832, 0.        , 0.00201416,
       0.46899414, 0.00201416, 0.00201416, 0.02197266, 0.04199219])

In [34]:
# are they equal?
np.array([ returned_dosages, calculated_dosages ]).T

array([[0.00100708, 0.00201416],
       [0.00100708, 0.00201416],
       [0.00201416, 0.00402832],
       [0.        , 0.        ],
       [0.00100708, 0.00201416],
       [       nan, 0.46899414],
       [0.00100708, 0.00201416],
       [0.00100708, 0.00201416],
       [0.01098633, 0.02197266],
       [0.02099609, 0.04199219]])

In [37]:
# apparently not... they look to be half of the I would expect
a = np.array( [ returned_dosages, calculated_dosages, calculated_dosages ]).T
a[:,0] / a[:,1]

<ipython-input-37-25f6b3bcdaed>:3: RuntimeWarning: invalid value encountered in true_divide
  a[:,0] / a[:,1]


array([0.5, 0.5, 0.5, nan, 0.5, nan, 0.5, 0.5, 0.5, 0.5])

Yup, and there's the NAs which I imagine might have something to do with
a probability threshold. Ok, let's look at pybgen code and try to understand...
https://lemieuxl.github.io/pybgen/_modules/pybgen/pybgen.html#PyBGEN

- there is a probability threshold, set by default to 0.9. The result
is that dosages where any of the probabilities is greater than 0.9 are returned
as NA

- Ok, the probability bit is a little confusing but I think we are ok... if I understand code
correctly, probs returned are PAA, PBB, PAB. In which case the calculation I performed 
was in fact incorrect... Let's try again...

In [38]:
def mydosage(probs):
    probs = np.array(probs)
    return(probs[1] + 2 * probs[2])

with PyBGEN(chr21_bgen, probs_only=True) as bgen:
    bgen.next()
    _, probs = bgen.next()
    probs = probs[:10]
    calculated_dosages = np.array([ mydosage(p) for p in probs ])
calculated_dosages

array([0.00100708, 0.00100708, 0.00201416, 0.        , 0.00100708,
       0.23898315, 0.00100708, 0.00100708, 0.01098633, 0.02099609])

In [39]:
# are they equal?
np.array([ returned_dosages, calculated_dosages ]).T

array([[0.00100708, 0.00100708],
       [0.00100708, 0.00100708],
       [0.00201416, 0.00201416],
       [0.        , 0.        ],
       [0.00100708, 0.00100708],
       [       nan, 0.23898315],
       [0.00100708, 0.00100708],
       [0.00100708, 0.00100708],
       [0.01098633, 0.01098633],
       [0.02099609, 0.02099609]])

Cool. I think we are good. Ok, let's give it a first try on how to write
a tsv file of data. 

In [78]:
from progress.bar import Bar

def write_dosage_tsv(bgen_file, output_file, sample_df, chr, num_variants=0, num_samples=0):
    colname_prefix = ['rsid', 'chr', 'pos', 'ref', 'alt']
    if num_samples == 0:
        num_samples = len(sample_df)
    
    assert 'ID_2' in sample_df.columns, "ID_2 column not found in sample_df"

    samplenames = list(sample_df.ID_2[:num_samples].values)
    colnames = colname_prefix + samplenames

    with open(output_file, 'w') as out, PyBGEN(bgen_file) as bgen:
        assert num_samples <= bgen.nb_samples, f"bgen has {bgen.nb_samples} samples and we want {num_samples}"
        
        out.write("\t".join(colnames) + "\n")
        
        if num_variants == 0:
            num_variants = bgen.nb_variants

        print(f"Reading bgen file {bgen_file}")
        print(f"Writing tsv file {output_file}")
        print(f"num variants: {num_variants}, num samples: {num_samples}")

        bar = Bar('Writing', max=num_variants)
        i = 0
        for info, dosage in bgen:
            if i == num_variants:
                break
            i += 1
            bar.next()

            # subset to the samples we want
            dosage = dosage[:num_samples]

            # dont print if they are all na
            if np.isnan(dosage).all():
                continue

            # dont print if they all have the same value
            if np.max(dosage) == np.min(dosage):
                continue

            info_str = f"{info.name}\t{chr}\t{info.pos}\t{info.a1}\t{info.a2}"
            dosage_str = "\t".join([ str(d) for d in dosage[:num_samples]])
            out.write(info_str + "\t" + dosage_str + "\n")
        bar.finish()



In [79]:
# alright let's test it out with 20 variants and 10 samples
write_dosage_tsv(chr21_bgen, "ADNI_merged_21_dosage.tsv", sample_df, "chr21", 20, 10)


Reading bgen file ADNI_merged_21.bgen
Writing tsv file ADNI_merged_21_dosage.tsv
num variants: 20, num samples: 10


In [80]:
# ok let's take a look
test_df = pd.read_csv("ADNI_merged_21_dosage.tsv", delimiter="\t")
test_df

,rsid,chr,pos,ref,alt,014_S_0520,005_S_1341,012_S_0803,018_S_0055,027_S_0118,027_S_0403,053_S_0389,041_S_0262,005_S_1224,010_S_0472
0,rs184649157,chr21,9576494,G,A,0.001007,0.001007,0.002014,0.000000,0.001007,NaN,0.001007,0.001007,0.010986,0.020996
1,rs191306879,chr21,9585758,G,C,0.003998,0.000000,0.000000,0.028992,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,rs201984519,chr21,9710110,TGA,T,0.001007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,rs142055579,chr21,9959950,A,G,NaN,0.006012,0.002991,0.002014,0.035004,0.002991,0.002991,0.032013,0.002014,0.002991
4,rs202178237,chr21,10031582,TA,T,0.000000,0.000000,0.001007,0.002014,0.001007,0.006989,0.000000,0.000000,0.010010,0.024994
5,rs150848991,chr21,10208211,C,T,0.001007,0.002014,0.003998,0.000000,NaN,0.001007,0.001007,0.002014,0.009003,0.000000


In [99]:
# ok, let's do this for chr21 now
write_dosage_tsv("ADNI_merged_21.bgen", "ADNI_merged_21_dosage.tsv", sample_df, "chr21")

Reading bgen file ADNI_merged_21.bgen
Writing tsv file ADNI_merged_21_dosage.tsv
num variants: 177285, num samples: 940


In [100]:
!wc -l ADNI_merged_21_dosage.tsv

  175823 ADNI_merged_21_dosage.tsv


In [101]:
chr21_dat = pd.read_csv("ADNI_merged_21_dosage.tsv", delimiter="\t", nrows=100)
chr21_dat

,rsid,chr,pos,ref,alt,014_S_0520,005_S_1341,012_S_0803,018_S_0055,027_S_0118,...,009_S_2381,068_S_4274,041_S_4014,031_S_4194,141_S_4423,011_S_2274,094_S_4282,033_S_4179,033_S_4176,099_S_2042
0,rs186660003,chr21,9419528,A,G,NaN,NaN,NaN,NaN,NaN,...,0.001007,0.0,0.000000,0.001007,NaN,0.002991,0.000000,0.000000,0.000000,0.002014
1,rs184649157,chr21,9576494,G,A,0.001007,0.001007,0.002014,0.000000,0.001007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rs191306879,chr21,9585758,G,C,0.003998,0.000000,0.000000,0.028992,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rs193227632,chr21,9675950,T,A,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,0.009003,0.049011,0.000000,0.002991,0.006012,0.000000,0.001007,0.000000
4,rs180808578,chr21,9680948,A,G,NaN,NaN,NaN,NaN,NaN,...,0.001007,0.0,0.002014,0.001007,0.001007,0.006989,0.001007,0.002014,0.001007,0.002014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,rs149722537,chr21,14404392,G,T,0.001007,0.001007,0.000000,0.009003,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,rs189994453,chr21,14404743,A,C,0.001007,0.001007,0.001007,0.009003,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,rs28673345,chr21,14404991,A,T,0.000000,0.000000,0.000000,0.001007,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
98,rs185263172,chr21,14407469,C,T,0.000000,0.000000,0.000000,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
# and now let's do chr19 
write_dosage_tsv("ADNI_merged_19.bgen", "ADNI_merged_19_dosage.tsv", sample_df, "chr19")

Reading bgen file ADNI_merged_19.bgen
Writing tsv file ADNI_merged_19_dosage.tsv
num variants: 293057, num samples: 940


In [103]:
!wc -l ADNI_merged_19_dosage.tsv


  292018 ADNI_merged_19_dosage.tsv


In [104]:
chr19_dat = pd.read_csv("ADNI_merged_19_dosage.tsv", delimiter="\t", nrows=100)
chr19_dat

,rsid,chr,pos,ref,alt,014_S_0520,005_S_1341,012_S_0803,018_S_0055,027_S_0118,...,009_S_2381,068_S_4274,041_S_4014,031_S_4194,141_S_4423,011_S_2274,094_S_4282,033_S_4179,033_S_4176,099_S_2042
0,rs201816663,chr19,80840,CCT,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.009003,NaN,NaN,NaN,NaN
1,rs3020701,chr19,90974,T,C,NaN,NaN,NaN,NaN,NaN,...,0.002014,0.0,0.006989,0.005005,0.000000,0.000000,0.0,0.000000,0.0,0.0
2,rs199586602,chr19,93818,GATTT,G,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.001007,0.009003,0.014008,0.000000,0.0,0.007996,0.0,0.0
3,rs56182540,chr19,95981,G,A,0.002014,0.036987,0.000000,0.0,0.002014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rs7260412,chr19,105021,G,C,0.000000,0.000000,0.000000,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,rs2312723,chr19,265083,G,A,0.000000,0.002014,0.000000,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,rs6510643,chr19,265445,G,A,1.000000,0.997986,1.002014,0.0,1.997986,...,1.993011,1.0,1.997986,0.998993,1.000000,2.000000,1.0,1.000000,2.0,2.0
97,rs77117568,chr19,265551,T,A,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
98,rs6510644,chr19,265662,T,C,1.000000,0.997986,1.002014,0.0,1.997986,...,2.000000,1.0,2.000000,0.998993,1.000000,2.000000,1.0,1.000000,2.0,2.0


In [105]:
# let's gzip them to reduce size
!gzip ADNI_merged_19_dosage.tsv

In [106]:
!gzip ADNI_merged_21_dosage.tsv

In [107]:
!ls -lh

total 1245760
-rw-r--r--@ 1 corradah  staff    31K May 21 07:45 ADNI_merged.sample
-rw-r--r--@ 1 corradah  staff   249M May 21 08:05 ADNI_merged_19.bgen
-rw-r--r--  1 corradah  staff    13M May 21 08:07 ADNI_merged_19.bgen.bgi
-rw-r--r--  1 corradah  staff   126M May 21 13:24 ADNI_merged_19_dosage.tsv.gz
-rw-r--r--@ 1 corradah  staff   118M May 19 08:45 ADNI_merged_21.bgen
-rw-r--r--  1 corradah  staff   7.3M May 21 07:09 ADNI_merged_21.bgen.bgi
-rw-r--r--  1 corradah  staff    64M May 21 13:22 ADNI_merged_21_dosage.tsv.gz
-rw-r--r--@ 1 corradah  staff    19M May 21 10:01 ADNI_merged_22.snp-stats
-rw-r--r--  1 corradah  staff    48K May 21 13:27 prep_data.ipynb
